# PyCUDA installation


In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=16164a743c2ec3994f3377bf47cb890ea0f33027cf32b54f0e66e5cb70608d06
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda




---



# Version #1: using ```SourceModule```

PyCUDA initialization

In [2]:
import numpy as np

# --- PyCUDA initialization
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


iDivUp function: if ```b``` divides ```a```, then ```a/b``` is returned, otherwise the function returns the integer division between ```a``` and ```b``` ```+1```.



In [3]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b): return (a + b - 1) // b

In [4]:
########
# MAIN #
########

Defining two CUDA events that will be used to measure execution time.

In [5]:
start = cuda.Event()
end   = cuda.Event()

Number of array elements

In [6]:
N = 100000

Number of threads per block

In [7]:
BLOCKSIZE = 256

Create two host vectors ```h_a``` and ```h_b``` of ```N``` random entries. ```np.random.randn``` returns ```float64```'s.

In [8]:
h_a = np.random.randn(1, N)
h_b = np.random.randn(1, N)

Cast ```h_a``` and ```h_b``` to single precision (```float32```).

In [9]:
h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)

Allocate ```h_a.nbytes```, ```h_b.nbytes``` and ```h_c.nbytes``` of GPU device memory space pointed to by ```d_a```, ```d_b``` and ```d_c```, respectively.

In [10]:
d_a = cuda.mem_alloc(h_a.nbytes)
d_b = cuda.mem_alloc(h_b.nbytes)
d_c = cuda.mem_alloc(h_a.nbytes)

Copy the ```h_a``` and ```h_b``` arrays from host to the device arrays ```d_a``` and ```d_b```, respectively.

In [11]:
cuda.memcpy_htod(d_a, h_a)
cuda.memcpy_htod(d_b, h_b)

Define the CUDA kernel function ```deviceAdd``` as a string. ```deviceAdd``` performs the elementwise summation of ```d_a``` and ```d_b``` and puts the result in ```d_c```.



In [12]:
mod = SourceModule("""
  #include <stdio.h>
  __global__ void deviceAdd(float * __restrict__ d_c, const float * __restrict__ d_a, const float * __restrict__ d_b, const int N)
  {
    const int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= N) return;
    d_c[tid] = d_a[tid] + d_b[tid];
  }
  """)

Define a reference to the ```__global__``` function ```deviceAdd```.

In [13]:
deviceAdd = mod.get_function("deviceAdd")

Define the block size ```blockDim``` and the grid size ```gridDim```.

In [14]:
blockDim  = (BLOCKSIZE, 1, 1)
gridDim   = (int(iDivUp(N, BLOCKSIZE)), 1, 1)

Invoke the ```deviceAdd``` function.
Note that, up to here, ```N``` is an *object* of ```class int``` and not an integer number. Therefore, before using it, we must cast it to ```np.int32``` which is essentially the standard, ```32```-bit integer type (matching CUDA’s ```int```).
Before launching ```deviceAdd```, the ```start``` and ```end``` events are recorded, so that the execution time can be measured.
Note that, before the processing time can be measured, all the activities in the current context must be ceased. This is the reason why ```end.synchronize()``` is invoked. Remember that the host and device executions are asynchronous. Without ```end.synchronize()```, the timing would reflect only the kernel launch latency, not its execution. Furthermore, with the event record, the device will record a time stamp for the event when it reaches that event in the stream. Without synchronization, it happens that the ```end``` event is recorded after the ```deviceAdd``` function execution is actually terminated, as we expect, but the ```print``` function is executed before ```deviceAdd``` has actually finished its execution.


In [15]:
# --- Warmup execution
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)

start.record()
deviceAdd(d_c, d_a, d_b, np.int32(N), block = blockDim, grid = gridDim)
end.record()
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

Processing time = 0.000268s


Allocate host space and copy results from device to host.

In [16]:
h_c = np.empty_like(h_a)
cuda.memcpy_dtoh(h_c, d_c)

Check if the device processing results are as expected.

In [17]:
if np.array_equal(h_c, h_a + h_b):
  print("Test passed!")
else :
  print("Error!")

Test passed!


Finally, flush context printf buffer. Without flushing, no printout may be returned.

In [18]:
cuda.Context.synchronize()